# ELECTRAのノートブック

## 設定

In [1]:
import os
import random
import numpy as np
import pandas as pd
import string
import unicodedata
from typing import Any, Dict, Iterator, List, Tuple, Union
import torch
import datasets
from datasets import load_metric


import transformers 
from transformers import AutoTokenizer,AlbertTokenizer,BertJapaneseTokenizer,BertJapaneseTokenizer,T5Tokenizer
from transformers import AutoModelForQuestionAnswering,TrainingArguments,Trainer,ElectraForMaskedLM,RobertaForMaskedLM,AutoModel
from transformers import default_data_collator
import os



In [2]:
from huggingface_hub import notebook_login
notebook_login()

### 変数

In [2]:
##ここを変更する
args = {
    'random_seed': 42,
    'pretrained_model': './model/bert-base-japanese-whole-word-masking_test/checkpoint-16781',
    'pretrained_tokenizer': 'cl-tohoku/bert-base-japanese-whole-word-masking',
    'batch_size': 16, 
    "eval_batch_size":16,
    'lr': 2e-5, 
    'max_length': 384,  
    'doc_stride': 128,  
    'epochs': 3,  
    'dataset': 'SkelterLabsInc/JaQuAD',
    'optimizer': 'AdamW',
    'norm_form': 'NFKC',
    'weight_decay': 0.01,  
    'lr_scheduler': 'warmup_lin',
    'warmup_ratio': 0.1,
    "eval_accumulation_steps":10,
    'cpu_workers': os.cpu_count(),
    'note':"same_prepare",
}
args

tokenizer = BertJapaneseTokenizer.from_pretrained(
            args["pretrained_tokenizer"]
            if args["pretrained_tokenizer"] else
            args["pretrained_model"])

model = AutoModelForQuestionAnswering.from_pretrained(args['pretrained_model'])


In [3]:
#seed値を固定
def set_seed(seed =42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic =True
set_seed(seed=args["random_seed"])

#wandb
import wandb
wandb.login()
os.environ["WANDB_PROJECT"] = "JaQuad"

#細かい設定
#!sudo apt install git-lfs
pad_on_right = tokenizer.padding_side == "right"
os.environ["TOKENIZERS_PARALLELISM"] = "false"
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

#モデルを軽量化
def quantize_transform(model):
    model = torch.quantization.quantize_dynamic(model, {torch.nn.Linear}, dtype=torch.qint8)
    return model
quantize_transform(model)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: cashunsukechiba (use `wandb login --relogin` to force relogin)


cuda:0


BertForQuestionAnswering(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(32000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): DynamicQuantizedLinear(in_features=768, out_features=768, dtype=torch.qint8, qscheme=torch.per_tensor_affine)
              (key): DynamicQuantizedLinear(in_features=768, out_features=768, dtype=torch.qint8, qscheme=torch.per_tensor_affine)
              (value): DynamicQuantizedLinear(in_features=768, out_features=768, dtype=torch.qint8, qscheme=torch.per_tensor_affine)
              (dropout): Dropout(p=0.1, inplace=False)
            )
           

## データセットの準備

### datasetdict

In [4]:
datasetdict = datasets.load_dataset(args['dataset'])
datasetdict = datasetdict.flatten()\
            .rename_column('answers.text', 'answer')\
            .rename_column('answers.answer_start', 'answer_start')\
            .rename_column('answers.answer_type', 'answer_type')

Using custom data configuration default
Reusing dataset ja_qu_ad (/home/s16991/.cache/huggingface/datasets/SkelterLabsInc___ja_qu_ad/default/0.1.0/5847b2e2ab5e02de284395bb15f87f13eae8f6f6ff1f01e4ee9c5c0dcf8ef8eb)


  0%|          | 0/2 [00:00<?, ?it/s]

### functions

In [5]:
#refer to official base line
def preprocess_function(examples):
        tokenized_examples = tokenizer(
            examples['question' if pad_on_right else "context"],
            examples['context' if pad_on_right else "question"],
            return_overflowing_tokens=True,
            padding="max_length",
        )

        inputs = {
            'input_ids': [],
            'attention_mask': [],
            'start_positions': [],
            'end_positions': [],
        }
        for tokens, att_mask, type_ids, context, answer,question,start_char \
                in zip(tokenized_examples['input_ids'],
                       tokenized_examples['attention_mask'],
                       tokenized_examples['token_type_ids'],
                       examples['context'],
                       examples['answer'],
                       examples['question'],
                       examples['answer_start']):

            answer = answer[0]
            start_char = start_char[0]
            offsets = get_offsets(tokens, context, tokenizer,
                                  args["norm_form"])
            

            ctx_start = tokens.index(tokenizer.sep_token_id) + 1
            answer_start_index = 0
            answer_end_index = len(offsets) - 2
            
            while offsets[answer_start_index][0] < start_char:
                answer_start_index += 1
            while offsets[answer_end_index][1] > start_char + len(answer):
                answer_end_index -= 1
            answer_start_index += ctx_start
            answer_end_index += ctx_start

            span_inputs = {
                'input_ids': tokens,
                'attention_mask': att_mask,
                'token_type_ids': type_ids,
            }

            for span, answer_idx in make_spans(
                span_inputs,
                question_len=ctx_start,
                max_seq_len=args["max_length"],
                stride=args["doc_stride"],
                answer_start_position=answer_start_index,
                answer_end_position=answer_end_index):
                inputs['input_ids'].append(span['input_ids'])
                inputs['attention_mask'].append(span['attention_mask'])
                inputs['start_positions'].append(answer_idx[0])
                inputs['end_positions'].append(answer_idx[1]) 
            
        return inputs
   
def make_spans(
    inputs: Dict[str, Union[int, List[int]]],
    question_len: int,
    max_seq_len: int,
    stride: int,
    answer_start_position: int = -1,
    answer_end_position: int = -1,
) -> Iterator[Tuple[Dict[str, List[int]], Tuple[int, int]]]:
    input_len = len(inputs['input_ids'])
    context_len = input_len - question_len

    def make_value(input_list, i, padding=0):
        context_end = min(max_seq_len - question_len, context_len - i)
        pad_len = max_seq_len - question_len - context_end
        val = input_list[:question_len]
        val += input_list[question_len + i:question_len + i + context_end]
        val[-1] = input_list[-1]
        val += [padding] * pad_len
        return val
    for i in range(0, input_len - max_seq_len + stride, stride):
        span = {key: make_value(val, i) for key, val in inputs.items()}
        answer_start = answer_start_position - i
        answer_end = answer_end_position - i
        if answer_start < question_len or answer_end >= max_seq_len - 1:
            answer_start = answer_end = 0
        yield span, (answer_start, answer_end)    

def get_offsets(input_ids: List[int],
                context: str,
                tokenizer: AutoTokenizer,
                norm_form='NFKC') -> List[Tuple[int, int]]:
    
    cxt_start = input_ids.index(tokenizer.sep_token_id) + 1
    cxt_end = cxt_start + input_ids[cxt_start:].index(tokenizer.sep_token_id)
    tokens = tokenizer.convert_ids_to_tokens(input_ids[cxt_start:cxt_end])
    tokens = [tok[2:] if tok.startswith('##') else tok for tok in tokens]
    whitespace = string.whitespace + '\u3000'

    # 1 . Make offsets of normalized context within the original context.
    offsets_norm_context = []
    norm_context = ''
    for idx, char in enumerate(context):
        norm_char = unicodedata.normalize(norm_form, char)
        norm_context += norm_char
        offsets_norm_context.extend([idx] * len(norm_char))
    norm_context_org = unicodedata.normalize(norm_form, context)
    assert norm_context == norm_context_org, \
        'Normalized contexts are not the same: ' \
        + f'{norm_context} != {norm_context_org}'
    assert len(norm_context) == len(offsets_norm_context), \
        'Normalized contexts have different numbers of tokens: ' \
        + f'{len(norm_context)} != {len(offsets_norm_context)}'

    # 2. Make offsets of tokens (input_ids) within the normalized context.
    offsets_token = []
    unk_pointer = None
    cid = 0
    tid = 0
    while tid < len(tokens):
        cur_token = tokens[tid]
        if cur_token == tokenizer.unk_token:
            unk_pointer = tid
            offsets_token.append([cid, cid])
            cid += 1
        elif norm_context[cid:cid + len(cur_token)] != cur_token:
            # Wrong offsets of the previous UNK token
            assert unk_pointer is not None, \
                'Normalized context and tokens are not matched'
            prev_unk_expected = offsets_token[unk_pointer]
            prev_unk_expected[1] += norm_context[prev_unk_expected[1] + 2:]\
                .index(tokens[unk_pointer + 1]) + 1
            tid = unk_pointer
            offsets_token = offsets_token[:tid] + [prev_unk_expected]
            cid = prev_unk_expected[1] + 1
        else:
            start_pos = norm_context[cid:].index(cur_token)
            if start_pos > 0 and tokens[tid - 1] == tokenizer.unk_token:
                offsets_token[-1][1] += start_pos
                cid += start_pos
                start_pos = 0
            assert start_pos == 0, f'{start_pos} != 0 (cur: {cur_token}'
            offsets_token.append([cid, cid + len(cur_token) - 1])
            cid += len(cur_token)
            while cid < len(norm_context) and norm_context[cid] in whitespace:
                offsets_token[-1][1] += 1
                cid += 1
        tid += 1
    if tokens[-1] == tokenizer.unk_token:
        offsets_token[-1][1] = len(norm_context) - 1
    else:
        assert cid == len(norm_context) == offsets_token[-1][1] + 1, \
            'Offsets do not include all characters'
    assert len(offsets_token) == len(tokens), \
        'The numbers of tokens and offsets are different'

    offsets_mapping = [(offsets_norm_context[start], offsets_norm_context[end])
                       for start, end in offsets_token]
    return [(0, 0)] + offsets_mapping+[(0, 0)] 

In [13]:
#テスト
examples = datasetdict['train'][:3]
tokenized_examples = tokenizer(
            examples['question' if pad_on_right else "context"],
            examples['context' if pad_on_right else "question"],
            return_overflowing_tokens=True,
            padding="max_length",
        )
# features = preprocess_function(datasetdict['train'][:3])
# print(features)
# len(features["input_ids"])
# for tokens in zip(tokenized_examples["input_ids"]):
#     print(tokens)
#     print(type(tokens[0]))
# tokenized_examples["input_ids"]
# tokenizer.sep_token_id
# tokens.index(tokenizer.sep_token_id) + 1
tokenized_examples

{'overflowing_tokens': [[], [], []], 'num_truncated_tokens': [-361, -370, -206], 'input_ids': [[2, 2343, 91, 5, 3335, 1430, 5, 18200, 12, 6, 4073, 2259, 5, 52, 635, 28, 31, 1430, 167, 9, 3654, 2935, 3, 14554, 23259, 23, 16, 26100, 73, 26, 28870, 6, 3326, 266, 14554, 1311, 23, 3137, 552, 222, 5083, 19, 23, 398, 48, 19, 24, 213, 37, 48, 32, 61, 2305, 19, 23, 593, 2152, 24, 25, 37, 159, 32, 24, 9, 6, 91, 5, 1430, 167, 6, 17902, 6, 1149, 734, 12, 31, 8, 2343, 91, 320, 3335, 1430, 5, 18200, 50, 6, 1430, 1774, 5, 5828, 104, 81, 18, 451, 50, 1298, 15, 10, 8, 3460, 149, 5703, 94, 750, 23, 8507, 9, 937, 1314, 1229, 28742, 417, 25126, 252, 6, 265, 5, 25126, 94, 24, 10105, 2801, 2270, 12, 31, 8, 937, 1443, 396, 5957, 4073, 1534, 129, 11, 912, 8, 3970, 4524, 2114, 1621, 4073, 2259, 23, 3574, 24119, 18502, 35, 3963, 19, 258, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [34]:
#all data
tokenized_train_dataset = datasetdict["train"].map(preprocess_function, batched=True, remove_columns=datasetdict["train"].column_names)
tokenized_valid_dataset = datasetdict["validation"].map(preprocess_function, batched=True, remove_columns=datasetdict["train"].column_names)
tokenized_train_dataset.set_format(type='torch')
tokenized_valid_dataset.set_format(type='torch')

  0%|          | 0/32 [00:00<?, ?ba/s]

  0%|          | 0/4 [00:00<?, ?ba/s]

## Fine-tuning the model

In [35]:
model_name = args['pretrained_model'].split("/")[-1]
note = args["note"]
data_collator = default_data_collator

train_args = TrainingArguments(
    output_dir= f"./model/{model_name}_{note}",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=args["lr"],
    per_device_train_batch_size= args["batch_size"],
    per_device_eval_batch_size= args["eval_batch_size"],
    num_train_epochs=args["epochs"],
    weight_decay=args["weight_decay"],
    push_to_hub=True,
    report_to="wandb",
    run_name=f"{model_name}_{note}",
    load_best_model_at_end = True,
    eval_accumulation_steps = args["eval_accumulation_steps"] #メモリ対策？
)

trainer =  Trainer(
    model,
    train_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_valid_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=None,
)

PyTorch: setting up devices
Cloning https://huggingface.co/chiba/checkpoint-16781_same_prepare into local empty directory.


In [10]:
trainer.train()

/opt/conda/lib/python3.7/site-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 67123
  Num Epochs = 3
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 12588
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


Epoch,Training Loss,Validation Loss
1,1.522300,1.526319
2,1.133500,1.362786
3,0.912200,1.334814


***** Running Evaluation *****
  Num examples = 8043
  Batch size = 16
Saving model checkpoint to ./model/electra-small-japanese-generator_same_prepare/checkpoint-4196
Configuration saved in ./model/electra-small-japanese-generator_same_prepare/checkpoint-4196/config.json
Model weights saved in ./model/electra-small-japanese-generator_same_prepare/checkpoint-4196/pytorch_model.bin
tokenizer config file saved in ./model/electra-small-japanese-generator_same_prepare/checkpoint-4196/tokenizer_config.json
Special tokens file saved in ./model/electra-small-japanese-generator_same_prepare/checkpoint-4196/special_tokens_map.json
tokenizer config file saved in ./model/electra-small-japanese-generator_same_prepare/tokenizer_config.json
Special tokens file saved in ./model/electra-small-japanese-generator_same_prepare/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 8043
  Batch size = 16
Saving model checkpoint to ./model/electra-small-japanese-generator_same_prepare/chec

TrainOutput(global_step=12588, training_loss=1.3776075934697076, metrics={'train_runtime': 3114.5245, 'train_samples_per_second': 64.655, 'train_steps_per_second': 4.042, 'total_flos': 4412628267876864.0, 'train_loss': 1.3776075934697076, 'epoch': 3.0})

## valuation

### function

In [36]:
def calculate_f1_score(start_positions, end_positions, start_preds,
                        end_preds):
    start_overlap = np.maximum(start_positions, start_preds)
    end_overlap = np.minimum(end_positions, end_preds)
    overlap = np.maximum(end_overlap - start_overlap + 1, 0)
    pred_token_count = np.maximum(end_preds - start_preds + 1, 0)
    ground_token_count = np.maximum(end_positions - start_positions + 1, 0)


    precision = torch.nan_to_num(overlap / pred_token_count, nan=0.)
    recall = torch.nan_to_num(overlap / ground_token_count, nan=0.)
    f1 = torch.nan_to_num(
        2 * precision * recall / (precision + recall), nan=0.)
    return {
        'precision': precision,
        'recall': recall,
        'f1': f1,
    }
    
def calculate_exact_match(start_positions, end_positions, start_preds,
                            end_preds):
    equal_start = (start_preds == start_positions)
    equal_end = (end_preds == end_positions)
    return (equal_start * equal_end).to('cpu').detach().numpy().astype(np.float32)

def get_result(dataset = tokenized_valid_dataset):
    results = trainer.predict(dataset)
    start_preds = results.predictions[0]
    end_preds = results.predictions[1]
    start_positions = results.label_ids[0]
    end_positions = results.label_ids[1]
    start_pred= torch.from_numpy(start_preds).argmax(dim=-1).cpu().detach()
    end_pred= torch.from_numpy(end_preds).argmax(dim=-1).cpu().detach()
    start_position = torch.from_numpy(start_positions).cpu().detach()
    end_position = torch.from_numpy(end_positions).cpu().detach()
    return start_pred,end_pred,start_position,end_position

In [37]:
start_pred,end_pred,start_position,end_position = get_result(tokenized_valid_dataset)
f1_metrics = calculate_f1_score(start_position,end_position,start_pred,end_pred)

precision = f1_metrics['precision'].mean()
recall = f1_metrics['recall'].mean()
f1 = f1_metrics['f1'].mean()
em = calculate_exact_match(start_position,end_position,start_pred,end_pred).mean()

print("========== CV ==========")
print("f1:",f1.item())
print("em:",em.item())

wandb.log({"f1": f1, "em": em})

***** Running Prediction *****
  Num examples = 8043
  Batch size = 16


========== CV ==========
f1: 0.8254448771476746
em: 0.7963446378707886


## Interface

In [43]:
#questions =["サイバーエージェントはいつ創業したの？","世の中にはどんな広告媒体があるの？","どうやって広告代理事業を行なっているの？","何を軸に事業を展開しているの？","サイバーエージェントは何から始まっている会社なの？"]
questions =["高橋 是清の職業は?","高橋 是清は立憲政友会の第何代総裁？","高橋 是清はなんと知られているの？","高橋 是清の愛称は？","高橋 是清の幼名は？"]
#questions =["どんな環境下で挑戦できるの？","最近はサーバサイドエンジニアも対象に何が行われているの？","最近はサーバサイドエンジニアも対象に何が実施されているの？","どんな組織風土があるの？","データサイエンスコンペティションはどんな形式？"]
#context = 'インターネットを軸に様々な事業を展開しております。 まず、サイバーエージェントは1998年に創業し、今年16周年目という節目にあたります。元々はBtoBといわれる対企業向けのビジネスとして、インターネットの広告代理事業から始まっている会社です。創業当初から行っており今も主力事業となっているうちの一つがインターネットの広告代理店の事業です。世の中には様々な広告の媒体があると思います。テレビCMもあれば、屋外広告、電車、雑誌、新聞、ラジオなどがある中で、私たちはインターネット専業で広告代理事業を行っています。総合広告代理店はTVを中心に全て扱うと思うのですが、私たちはその中でインターネットをメインとした専業でやっています。ここがBtoBといわれる対企業向けのビジネスとしての事業内容です'
context = '高橋 是清（たかはし これきよ、1854年9月19日〈嘉永7年閏7月27日〉 - 1936年〈昭和11年〉2月26日）は、明治から昭和にかけての日本の財政家、日銀総裁、政治家[1]。立憲政友会第4代総裁。第20代内閣総理大臣（在任: 1921年〈大正10年〉11月13日 - 1922年〈大正11年〉6月12日）。栄典は正二位大勲位子爵。幼名は和喜次（わきじ）。近代日本を代表する財政家として知られ、総理大臣としてよりも大蔵大臣としての評価の方が高い。愛称は「ダルマさん」。'
#context ='継続して価値を提供するためにチームとしての力を最大化して開発しようという組織風土があります。最新の情報キャッチアップできるようGoogle I/OやAWS re:Invent、国際学会などの海外で開催されるカンファレンスへの参加制度、勉強会やゼミ制度などがあり、切磋琢磨できる環境があります。最近ではサーバサイドエンジニアも対象とした機械学習ハンズオンやKaggle形式のデータサイエンスコンペティションなども実施されています。日々最新技術やマーケット情報が飛び交う環境下で技術感度の高いエンジニアと一緒にこの大きな時代の転換期にこそできる挑戦をしませんか？'
def predict_answer(question, context):
    inputs = tokenizer(
    question, context, add_special_tokens=True, return_tensors="pt")
    inputs.to(device)
    input_ids = inputs["input_ids"].tolist()[0]
    outputs = model(**inputs)
    answer_start_scores = outputs.start_logits
    answer_end_scores = outputs.end_logits
    # Get the most likely beginning of answer with the argmax of the score.
    answer_start = torch.argmax(answer_start_scores)-1

    answer_end = torch.argmax(answer_end_scores) -1
    # Get the most likely end of answer with the argmax of the score.
    # 1 is added to `answer_end` because the index pointed by score is inclusive.
    answer = tokenizer.convert_tokens_to_string(
        tokenizer.convert_ids_to_tokens(input_ids[answer_start:answer_end]))
    print("質問",question)
    print("answer",answer)
    print("answer_start",answer_start)
    print("answer_end",answer_end)
    
for i in range(len(questions)):
    question = questions[i]
    predict_answer(question, context)
    

質問 高橋 是清の職業は?
answer 財政 家
answer_start tensor(58, device='cuda:0')
answer_end tensor(60, device='cuda:0')
質問 高橋 是清は立憲政友会の第何代総裁？
answer 第 4
answer_start tensor(79, device='cuda:0')
answer_end tensor(81, device='cuda:0')
質問 高橋 是清はなんと知られているの？
answer 
answer_start tensor(140, device='cuda:0')
answer_end tensor(67, device='cuda:0')
質問 高橋 是清の愛称は？
answer 「 ダルマ さん
answer_start tensor(159, device='cuda:0')
answer_end tensor(162, device='cuda:0')
質問 高橋 是清の幼名は？
answer 和 喜次
answer_start tensor(124, device='cuda:0')
answer_end tensor(127, device='cuda:0')


In [ ]:
inputs = tokenizer(
    question, context, add_special_tokens=True, return_tensors="pt")
inputs.to(device)
input_ids = inputs["input_ids"].tolist()[0]
outputs = model(**inputs)
answer_start_scores = outputs.start_logits
answer_end_scores = outputs.end_logits

In [ ]:
# Get the most likely beginning of answer with the argmax of the score.
answer_start = torch.argmax(answer_start_scores)-1

answer_end = torch.argmax(answer_end_scores) -1
# Get the most likely end of answer with the argmax of the score.
# 1 is added to `answer_end` because the index pointed by score is inclusive.
answer = tokenizer.convert_tokens_to_string(
    tokenizer.convert_ids_to_tokens(input_ids[answer_start:answer_end]))
print(answer)
print(answer_start)
print(answer_end)